In [1]:
#!pip install chesslab

In [2]:
from sklearn.model_selection import train_test_split
from chesslab.utils import load_pkl
from chesslab.training_torch import fitting
import torch
import torch.nn as nn
import numpy as np

In [3]:
example=1
lr = 0.1
epochs=30
batch_size = 128
test_percent=0.1
path = 'D:/database/ccrl/'
name_data='ccrl_states_elo2.pkl'
name_labels='ccrl_results_elo2.pkl'
save_name='./tmp/torch_weights.0'
num_workers=4
loss_fn=nn.CrossEntropyLoss(reduction='mean')
optim=torch.optim.SGD

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_1={
    '.':torch.tensor([0 , 0, 0], dtype=torch.float),
    'p':torch.tensor([0 , 0, 1], dtype=torch.float),
    'P':torch.tensor([0 , 0,-1], dtype=torch.float),
    'b':torch.tensor([0 , 1, 0], dtype=torch.float),
    'B':torch.tensor([0 ,-1, 0], dtype=torch.float),
    'n':torch.tensor([1 , 0, 0], dtype=torch.float),
    'N':torch.tensor([-1, 0, 0], dtype=torch.float),
    'r':torch.tensor([0 , 1, 1], dtype=torch.float),
    'R':torch.tensor([0 ,-1,-1], dtype=torch.float),
    'q':torch.tensor([1 , 0, 1], dtype=torch.float),
    'Q':torch.tensor([-1, 0,-1], dtype=torch.float),
    'k':torch.tensor([1 , 1, 0], dtype=torch.float),
    'K':torch.tensor([-1,-1, 0], dtype=torch.float)
}

encoding_2={
    '.':torch.tensor([0,0,0,0],dtype=torch.float),
    'p':torch.tensor([1,0,0,0],dtype=torch.float),
    'P':torch.tensor([0,0,0,1],dtype=torch.float),
    'b':torch.tensor([0,1,0,0],dtype=torch.float),
    'B':torch.tensor([0,0,1,0],dtype=torch.float),
    'n':torch.tensor([1,1,0,0],dtype=torch.float),
    'N':torch.tensor([0,0,1,1],dtype=torch.float),
    'r':torch.tensor([1,0,1,0],dtype=torch.float),
    'R':torch.tensor([0,1,0,1],dtype=torch.float),
    'q':torch.tensor([1,0,0,1],dtype=torch.float),
    'Q':torch.tensor([0,1,1,0],dtype=torch.float),
    'k':torch.tensor([1,1,1,0],dtype=torch.float),
    'K':torch.tensor([0,1,1,1],dtype=torch.float)
}

In [6]:
class Model_1(nn.Module):

    def __init__(self):
        super().__init__()
        self.func_1=nn.ELU()
        self.func_2=nn.ELU()
        self.func_3=nn.ELU()
        self.func_4=nn.ELU()
        
        self.cnn_1 = nn.Conv2d(3, 32, kernel_size=7,padding=3)
        self.cnn_2 = nn.Conv2d(32, 64, kernel_size=5,padding=2)
        self.cnn_3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)

        self.linear_1 = nn.Linear(8*8*128,256 )
        self.linear_2 = nn.Linear(256, 2)

    def forward(self, x ):
        out = self.cnn_1(x)
        out = self.func_1(out)
        out = self.cnn_2(out)
        out = self.func_2(out)
        out = self.cnn_3(out)
        out = self.func_3(out)
        out = out.reshape([x.size(0), -1])
        out = self.linear_1(out)
        out = self.func_4(out)
        out = self.linear_2(out)

        return out
    
class Model_2(nn.Module):

    def __init__(self):
        super().__init__()
        self.func_1=nn.ReLU()
        self.func_2=nn.ReLU()
        self.func_3=nn.ReLU()
        self.func_4=nn.ReLU()
        
        self.cnn_1 = nn.Conv2d(4, 32, kernel_size=7,padding=3)
        self.cnn_2 = nn.Conv2d(32, 64, kernel_size=5,padding=2)
        self.cnn_3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)

        self.linear_1 = nn.Linear(8*8*128,256 )
        self.linear_2 = nn.Linear(256, 2)

    def forward(self, x ):
        out = self.cnn_1(x)
        out = self.func_1(out)
        out = self.cnn_2(out)
        out = self.func_2(out)
        out = self.cnn_3(out)
        out = self.func_3(out)
        out = out.reshape([x.size(0), -1])
        out = self.linear_1(out)
        out = self.func_4(out)
        out = self.linear_2(out)

        return out

In [7]:
np.random.seed(0)
torch.manual_seed(0)

x_data = load_pkl(path+name_data)
y_data = load_pkl(path+name_labels)[:,1] #Nota: las etiquetas deben de ser enteros, no onehot

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(4829155, 64)
(4829155,)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if example==1:
    model = Model_1().to(device)
    encoding=encoding_1
else:
    model = Model_2().to(device)
    encoding=encoding_2

print(device)
print(model)

cuda
Model_1(
  (func_1): ELU(alpha=1.0)
  (func_2): ELU(alpha=1.0)
  (func_3): ELU(alpha=1.0)
  (func_4): ELU(alpha=1.0)
  (cnn_1): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (cnn_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear_1): Linear(in_features=8192, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=2, bias=True)
)


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        device=device,
        model=model,
        optim=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding,
        num_workers=num_workers)

2021-10-25 02:59:27


Epoch: 01/30 | time: 210s = 3.5m | train loss: 0.5440 | train acc: 0.7111 | test loss: 0.5151 | test acc: 0.7313

Epoch: 02/30 | time: 209s = 3.5m | train loss: 0.4983 | train acc: 0.7441 | test loss: 0.4749 | test acc: 0.7599

Epoch: 03/30 | time: 208s = 3.5m | train loss: 0.4696 | train acc: 0.7636 | test loss: 0.4544 | test acc: 0.7731

Epoch: 04/30 | time: 208s = 3.5m | train loss: 0.4467 | train acc: 0.7782 | test loss: 0.4392 | test acc: 0.7827

...

Epoch: 26/30 | time: 205s = 3.4m | train loss: 0.1613 | train acc: 0.9403 | test loss: 0.3341 | test acc: 0.8907

Epoch: 27/30 | time: 205s = 3.4m | train loss: 0.1603 | train acc: 0.9411 | test loss: 0.3539 | test acc: 0.8931

Epoch: 28/30 | time: 204s = 3.4m | train loss: 0.1618 | train acc: 0.9408 | test loss: 0.3563 | test acc: 0.8950

Epoch: 29/30 | time: 205s = 3.4m | train loss: 0.1602 | train acc: 0.9417 | test loss: 0.3998 | test acc: 0.8825

Epoch: 30/30 | time: 204s = 3.4m | train loss: 0.1611 | train acc: 0.9417 | test loss: 0.3736 | test acc: 0.8937

fitting(epochs=1,
      x_train=x_train,
      y_train=y_train,
      x_test=x_test,
      y_test=y_test,
      device=device,
      model= model, 
      load_name='tmp/test_elo.0.29.pt',
      save_name=save_name,
      num_workers=num_workers)